In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("C:/Users/mamun/OneDrive/Desktop/Data_science_project/ML Zoomcamp/bank.csv",sep=',')
df.head()

### Question 1
* Install Pipenv
* What's the version of pipenv you installed?
* Use --version to find out

In [34]:
!pipenv --version

pipenv, version 2023.10.3


### Question 2
* Use Pipenv to install Scikit-Learn version 1.3.1
* What's the first hash for scikit-learn you get in Pipfile.lock?
* Note: you should create an empty folder for homework and do it there.

In [35]:
!pipenv install scikit-learn==1.3.1

Installing scikit-learn==1.3.1...
Resolving scikit-learn==1.3.1...
[    ] Installing...
Installation Succeeded
[    ] Installing scikit-learn...
[    ] Installing scikit-learn...

Installing dependencies from Pipfile.lock (98d9d9)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.


first hash for scikit-learn : sha256": "e0d12a0331eb3926e2e7c7c37be9118ab3619b518475b3c3897cdb40da98d9d9"

### Question - 3

PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework
* wget $PREFIX/model1.bin
* wget $PREFIX/dv.bin
* Write a script for loading these models with pickle
* Score this client:
{"job": "retired", "duration": 445, "poutcome": "success"}
* What's the probability that this client will get a credit?

In [36]:
import requests

# Define the URL for the files
url_prefix = "https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework"
model_url = f"{url_prefix}/model1.bin"
dv_url = f"{url_prefix}/dv.bin"

# local file names to save the downloaded files
model_filename = "model1.bin"
dv_filename = "dv.bin"

# Download the model1.bin file
response = requests.get(model_url)
with open(model_filename, 'wb') as file:
    file.write(response.content)

# Download the dv.bin file
response = requests.get(dv_url)
with open(dv_filename, 'wb') as file:
    file.write(response.content)


In [37]:
import pickle

# Load the scikit-learn model
with open('model1.bin', 'rb') as model_file:
    scikit_model = pickle.load(model_file)

# Load the label encoder (dv.bin) to transform input data
with open('dv.bin', 'rb') as encoder_file:
    dv = pickle.load(encoder_file)

# Define the input data
input_data = {
    "job": "retired",
    "duration": 445,
    "poutcome": "success"
}

# Transform the input data using the label encoder
input_data_transformed = dv.transform([input_data])

# Make a probability prediction
probability = scikit_model.predict_proba(input_data_transformed)

# Get the probability of getting a credit (class 1)
probability_of_getting_credit = probability[0][1]

print(f"Probability of getting credit: {probability_of_getting_credit:.3f}")


Probability of getting credit: 0.902


### Question-4
Now let's serve this model as a web service
* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using requests:
* url = "YOUR_URL"
* client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
requests.post(url, json=client).json()

In [52]:
!pip install Flask gunicorn


In [55]:
import pickle

from flask import Flask
from flask import request
from flask import jsonify


def load_bin(filename: str):
    with open(filename, 'rb') as f_in:
        return pickle.load(f_in)


dv = load_bin('dv.bin')
model = load_bin('model1.bin')

app = Flask('bank-credict-scoring')


@app.route('/predict', methods=['POST'])
def predict():
    client = request.get_json()

    X = dv.transform([client])
    y_pred = model.predict_proba(X)[0][1]

    result = {
        'get_credit_probability': float(y_pred)
    }

    return jsonify(result)


if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)

 * Serving Flask app 'bank-credict-scoring'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:9696
 * Running on http://192.168.4.20:9696
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [54]:
import requests

url = "http://localhost:9696/predict"

client = {"job": "unknown", "duration": 270, "poutcome": "failure"}

response = requests.post(url, json=client).json()

print(response)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [43]:
requests.post(url,json=customer)

<Response [500]>

In [44]:
customer

{'job': 'unknown', 'duration': 270, 'poutcome': 'failure'}